In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd /content/gdrive/MyDrive/SemEval2023/SemEval2022-Task10/re_run

/content/gdrive/.shortcut-targets-by-id/1lC-ZKLaCDQyfLcof2Ak7FDa6IvTt318A/SemEval2023/SemEval2022-Task10/re_run


In [3]:
# !git pull origin main

In [4]:
import pandas as pd
# unsupervised_data_gab = pd.read_csv("Data/starting_ki/gab_1M_unlabelled.csv")
# unsupervised_data_reddit = pd.read_csv("Data/starting_ki/reddit_1M_unlabelled.csv")

In [7]:
from sklearn.model_selection import train_test_split

target_column = "label_category"
data = pd.read_csv("../Data/starting_ki/train_all_tasks.csv")

In [8]:
# data

In [9]:
data = data[data[target_column]!="none"]

In [10]:
if target_column == "label_category":
  label_values = [
      '1. threats, plans to harm and incitement',
      '2. derogation',
      '3. animosity',
      '4. prejudiced discussions',
  ]

#   label_map = {
#       0: '1. threats, plans to harm and incitement',
#       1: '2. derogation', 
#       2: '3. animosity', 
#       3: '4. prejudiced discussions',
#       '1. threats, plans to harm and incitement':0,
#       '2. derogation':1,
#       '3. animosity':2,
#       '4. prejudiced discussions':3,
#   }
elif target_column == "label_vector":
  label_values = [
      '1.1 threats of harm',
      '1.2 incitement and encouragement of harm',
      '2.1 descriptive attacks',
      '2.2 aggressive and emotive attacks',
      '2.3 dehumanising attacks & overt sexual objectification',
      '3.1 casual use of gendered slurs, profanities, and insults',
      '3.2 immutable gender differences and gender stereotypes',
      '3.3 backhanded gendered compliments',
      '3.4 condescending explanations or unwelcome advice',
      '4.1 supporting mistreatment of individual women',
      '4.2 supporting systemic discrimination against women as a group',
  ]

#   label_map = {
#       0: '1.1 threats of harm',
#       1: '1.2 incitement and encouragement of harm',
#       2: '2.1 descriptive attacks',
#       3: '2.2 aggressive and emotive attacks',
#       4: '2.3 dehumanising attacks & overt sexual objectification',
#       5: '3.1 casual use of gendered slurs, profanities, and insults',
#       6: '3.2 immutable gender differences and gender stereotypes',
#       7: '3.3 backhanded gendered compliments',
#       8: '3.4 condescending explanations or unwelcome advice',
#       9: '4.1 supporting mistreatment of individual women',
#       10: '4.2 supporting systemic discrimination against women as a group',
#       '1.1 threats of harm': 0,
#       '1.2 incitement and encouragement of harm': 1,
#       '2.1 descriptive attacks': 2,
#       '2.2 aggressive and emotive attacks': 3,
#       '2.3 dehumanising attacks & overt sexual objectification': 4,
#       '3.1 casual use of gendered slurs, profanities, and insults': 5,
#       '3.2 immutable gender differences and gender stereotypes': 6,
#       '3.3 backhanded gendered compliments': 7,
#       '3.4 condescending explanations or unwelcome advice': 8,
#       '4.1 supporting mistreatment of individual women': 9,
#       '4.2 supporting systemic discrimination against women as a group': 10,
#   }
# else:
#   raise Exception("Unknown label column")

## BERTweet

In [11]:
!pip install -q transformers

In [12]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
import torch
from transformers import AutoModel, AutoTokenizer, AutoModelForMaskedLM

# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# model = RobertaForMaskedLM.from_pretrained('roberta-base')

In [13]:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)
model = AutoModelForMaskedLM.from_pretrained("vinai/bertweet-base")

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
# tokenizer

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [16]:
!pip3 install -q nltk emoji==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 KB 246.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [17]:
from emoji import demojize
from nltk.tokenize import TweetTokenizer


tweettokenizer = TweetTokenizer()


def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token


def normalizeTweet(tweet):
    tokens = tweettokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = (
        normTweet.replace("cannot ", "can not ")
        .replace("n't ", " n't ")
        .replace("n 't ", " n't ")
        .replace("ca n't", "can't")
        .replace("ai n't", "ain't")
    )
    normTweet = (
        normTweet.replace("'m ", " 'm ")
        .replace("'re ", " 're ")
        .replace("'s ", " 's ")
        .replace("'ll ", " 'll ")
        .replace("'d ", " 'd ")
        .replace("'ve ", " 've ")
    )
    normTweet = (
        normTweet.replace(" p . m .", "  p.m.")
        .replace(" p . m ", " p.m ")
        .replace(" a . m .", " a.m.")
        .replace(" a . m ", " a.m ")
    )

    return " ".join(normTweet.split())

In [18]:
normalizeTweet("DHEC confirms https://postandcourier.com/health/covid19/sc-has-first-two-presumptive-cases-of-coronavirus-dhec-confirms/article_bddfe4ae-5fd3-11ea-9ce4-5f495366cee6.html?utm_medium=social&utm_source=twitter&utm_campaign=user-share… via @postandcourier 😢")


'DHEC confirms HTTPURL ... via @USER :crying_face:'

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
def get_prediction (sent, bestn=3):
    token_ids = tokenizer.encode(sent, return_tensors='pt')
    masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero()
    masked_pos = [mask.item() for mask in masked_position ]

    token_ids = token_ids.to(device)
    with torch.no_grad():
        output = model(token_ids)

    last_hidden_state = output[0].squeeze()

    list_of_list =[]
    for index,mask_index in enumerate(masked_pos):
        mask_hidden_state = last_hidden_state[mask_index]
        idx = torch.topk(mask_hidden_state, k=5, dim=0)[1]
        words = [tokenizer.decode(i.item()).strip().replace(" ", "") for i in idx]
        list_of_list.append(words)
        # print ("Mask ",index+1,"Guesses : ",words)
    
    best = []
    for i in range(bestn):
      best_guess = ""
      for j in list_of_list:
          best_guess = best_guess+" "+j[i]
      best.append(best_guess)
    return best

sentence = "Tom has fully ___ ___ ___ illness."
print ("Original Sentence: ",sentence)
sentence = sentence.replace("___","<mask>")
print ("Original Sentence replaced with mask: ",sentence)
print ("\n")

predicted_blanks = get_prediction(sentence)
print ("\nBest guess for fill in the blank :::",predicted_blanks)

Original Sentence:  Tom has fully ___ ___ ___ illness.
Original Sentence replaced with mask:  Tom has fully <mask> <mask> <mask> illness.



Best guess for fill in the blank ::: [' recovered from his', ' healed his mental', ' developed of this']


## Fill in the blank 

In [24]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Initialize the stopwords
stoplist = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [28]:
# ls Results/TaskB

In [27]:
import json

with open('Results/TaskB/lexicon_shapley.json') as fin:
    lexicon_shapley = json.load(fin)

In [30]:
from tqdm import tqdm


generated_sents = {}
for label in label_values:
  d = data[data[target_column]==label]

  lexicons = lexicon_shapley[label]
  generated_sents[label] = []
  for idx, row in tqdm(d.iterrows(), total=len(d)):
    text = normalizeTweet(row["text"])

    words = nltk.word_tokenize(text)
    widxs = []
    for widx, w in enumerate(words):
      if w in lexicons:
        widxs.append(widx)
    
    if len(widxs)==0:
      continue
    
    
    for widx in widxs:
      for n in range(1,4):
        newwords = [w if i!=widx else ("<mask> "*n).strip() for i, w in enumerate(words)]
        sentence = " ".join(newwords)
        predicted_blanks = get_prediction(sentence)

        for p in predicted_blanks:
          newsent = [w if i!=widx else p for i, w in enumerate(words)]

          generated_sents[label].append((words[widx], p, " ".join(newsent)))

100%|██████████| 333/333 [00:18<00:00, 18.00it/s]


## Filter the generated sentences

In [31]:
from transformers import BertTokenizer
import torch
from transformers import BertForSequenceClassification, AdamW, BertConfig
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

out_dir = f'../Models/1a_fine-tuned-bert'
model = BertForSequenceClassification.from_pretrained(out_dir)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


In [32]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

def get_input_tensor(texts):
  text_ids = []
  for text in tqdm(texts, total=len(texts)):
    ids = tokenizer.encode(text, max_length=100, padding='max_length', truncation=True)
    text_ids.append(ids)
  
  att_masks = []
  for ids in text_ids:
      masks = [int(id > 0) for id in ids]
      att_masks.append(masks)
  
  text_ids = torch.tensor(text_ids);
  att_masks = torch.tensor(att_masks);

  return text_ids, att_masks


filtered_sentences = {}
batch_size = 32
for label in generated_sents:
  filtered_sentences[label] = []
  texts = [t for _, _, t in generated_sents[label]]
  text_ids, att_masks = get_input_tensor(texts)
  
  test_data = TensorDataset(text_ids, att_masks)
  test_sampler = SequentialSampler(test_data)
  test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

  outputs = []
  with torch.no_grad():
      model.eval()
      for k, (mb_x, mb_m) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
          mb_x = mb_x.to(device)
          mb_m = mb_m.to(device)
          output = model(mb_x, attention_mask=mb_m)
          outputs.append(output[0].to('cpu'))

  outputs = torch.cat(outputs)

  _, predicted_values = torch.max(outputs, 1)
  predicted_values = predicted_values.numpy()
  predicted_labels = ["sexist" if p==1 else "not sexist" for p in predicted_values]

  for d, p in zip(generated_sents[label], predicted_labels):
    if p=="sexist":
      filtered_sentences[label].append(d)
  # break

100%|██████████| 145/145 [00:26<00:00,  5.38it/s]


In [39]:
filtered_sentences.keys()

dict_keys(['1. threats, plans to harm and incitement', '2. derogation', '3. animosity', '4. prejudiced discussions'])

In [33]:
filtered_sentences['1. threats, plans to harm and incitement'][0]

('women',
 ' women',
 '[ USER ] Yeah , liberal  women are so dumb they almost deserve the destruction that would befall them if they were thrown among muslims . Liberal women are about hopeless , they really are .')

In [34]:
import pandas as pd
df = []
for label in filtered_sentences:
  for w, nw, sent in filtered_sentences[label]:
    df.append({
        "text": sent,
        "label": label,
        "org_word": w,
        "new_word": nw
    })

df = pd.DataFrame(df)

In [35]:
df

,text,label,org_word,new_word
0,"[ USER ] Yeah , liberal women are so dumb the...","1. threats, plans to harm and incitement",women,women
1,"[ USER ] Yeah , liberal feminists are so dumb...","1. threats, plans to harm and incitement",women,feminists
2,"[ USER ] Yeah , liberal men are so dumb they ...","1. threats, plans to harm and incitement",women,men
3,"[ USER ] Yeah , liberal women women are so du...","1. threats, plans to harm and incitement",women,women women
4,"[ USER ] Yeah , liberal white feminists are s...","1. threats, plans to harm and incitement",women,white feminists
...,...,...,...,...
25496,"Never forget , it 's always the man 's fault ....",4. prejudiced discussions,tingles,free choice
25497,"Never forget , it 's always the man 's fault ....",4. prejudiced discussions,tingles,moral excuse
25498,"Never forget , it 's always the man 's fault ....",4. prejudiced discussions,tingles,time of wants
25499,"Never forget , it 's always the man 's fault ....",4. prejudiced discussions,tingles,choice to want


In [37]:
df.to_csv("task_b_fill_in_blank.csv", index=False)

In [50]:
ls

 Data/
 Models/
 Results/
'Split Data.ipynb'
'TaskA Fine-tuning BERT.ipynb'
'TaskA Fine-tuning BERTweet.ipynb'
'TaskA Fine-tuning TwHINBERT.ipynb'
'TaskB Augmentation with GPT-J.ipynb'
'Task B Augmentation w  PMI.ipynb'
'TaskB Augmentation w  XAI.ipynb'
'TaskB Lexicon Construciton [PMI].ipynb'
'TaskB Lexicon Construciton [Shapley].ipynb'
'TaskC Augmentation w  XAI.ipynb'
'TaskC Lexicon Construciton [PMI].ipynb'
'TaskC Lexicon Construciton [Shapley].ipynb'


## To Lexicons

In [51]:
import pandas as pd
data = pd.read_csv("task_b_fill_in_blank.csv")

In [94]:
# filtered_sentences = {}
# for label in label_values:
#   filtered_sentences[label] = []

# for idx, row in df.iterrows():
#   label = row["label"]
#   filtered_sentences[label].append((row["org_word"], row["new_word"], row["text"]))

In [95]:
# import json

# with open('../Results/TaskC/lexicon_shapley.json') as fin:
#     lexicon_shapley = json.load(fin)
    

In [96]:
# new_lexicon_shapley = {}
# for label in lexicon_shapley:
  

#   lexicons = lexicon_shapley[label]
#   sents = filtered_sentences[label]
#   new_lexicon = {}

#   for w, nw, sent in sents:
#     # skip multiple word
#     if " " in nw.strip():
#       continue
    
#     new_lexicon[nw] = 0.8*lexicons[w]

#   new_lexicon_shapley[label] = {}
#   for word in lexicons:
#     new_lexicon_shapley[label][word] = lexicons[word]

#   for word in new_lexicon:
#     # Ignore duplicated word from original lexicons
#     if word in new_lexicon_shapley[label]:
#       continue
#     new_lexicon_shapley[label][word] = new_lexicon[word]

In [97]:
# with open('../Results/TaskC/lexicon_shapley_augmented_v4_tweetbert.json', 'w') as outfile:
#     json.dump(new_lexicon_shapley, outfile)

## Calculate Shapley Values

In [80]:
from transformers import BertTokenizer
swtokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [54]:
import torch
from transformers import BertForSequenceClassification, AdamW, BertConfig

out_dir = f'../Models/1a_fine-tuned-bert'
model = BertForSequenceClassification.from_pretrained(out_dir)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


In [55]:
import transformers
import torch
import numpy as np
import scipy as sp

In [56]:
!pip install -q shap

In [57]:
import shap
# define a prediction function
def f(texts):
  text_ids = [tokenizer.encode(text, max_length=100, padding='max_length', truncation=True) for text in texts]

  att_masks = []
  for ids in text_ids:
      masks = [int(id > 0) for id in ids]
      att_masks.append(masks)

  text_ids = torch.tensor(text_ids).to(device)
  att_masks = torch.tensor(att_masks).to(device)

  outputs = model(text_ids, attention_mask=att_masks)
  outputs = outputs[0].detach().cpu().numpy()
  scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
  
  val = sp.special.logit(scores[:,1]) # use one vs rest logit units
  return val

In [58]:
import pickle

def save_shap_values(filepath, obj):
  with open(filepath, 'wb') as fin:
    pickle.dump(obj, fin)

In [59]:
explainer = shap.Explainer(f, tokenizer)

d = data
print("#Rows", len(d))
d = {"text": d["text"].values}

shap_values = explainer(d, fixed_context=1, batch_size=256)
save_shap_values(f"Results/shapley_values_bertweet_task_b.pickle", shap_values)

#Rows 25501


Partition explainer: 25502it [3:59:05,  1.78it/s]


In [65]:
# len(data), len(shap_values.values)

(25501, 25501)

In [66]:
data

,text,label,org_word,new_word
0,"[ USER ] Yeah , liberal women are so dumb the...","1. threats, plans to harm and incitement",women,women
1,"[ USER ] Yeah , liberal feminists are so dumb...","1. threats, plans to harm and incitement",women,feminists
2,"[ USER ] Yeah , liberal men are so dumb they ...","1. threats, plans to harm and incitement",women,men
3,"[ USER ] Yeah , liberal women women are so du...","1. threats, plans to harm and incitement",women,women women
4,"[ USER ] Yeah , liberal white feminists are s...","1. threats, plans to harm and incitement",women,white feminists
...,...,...,...,...
25496,"Never forget , it 's always the man 's fault ....",4. prejudiced discussions,tingles,free choice
25497,"Never forget , it 's always the man 's fault ....",4. prejudiced discussions,tingles,moral excuse
25498,"Never forget , it 's always the man 's fault ....",4. prejudiced discussions,tingles,time of wants
25499,"Never forget , it 's always the man 's fault ....",4. prejudiced discussions,tingles,choice to want


In [68]:
def get_lexicons(train, shap_values):
  lexicons = {}
  for idx, label in enumerate(label_values):
    d = train[train["label"]==label]
    s = shap_values.abs[d.index.to_numpy()]

    feature_names = s.mean(0).feature_names
    shapley_values = s.mean(0).values

    sorted_values = sorted(zip(shapley_values, feature_names), key=lambda pair: -pair[0])
    lexicons[label] = {x:v for v, x in sorted_values}
  return lexicons

aug_lexicons = get_lexicons(data, shap_values)

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


## Handle Subwords

In [69]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [70]:
from nltk.tokenize import TweetTokenizer
nltktokenizer = TweetTokenizer()

def word_tokenize(sent):
  # return nltk.word_tokenize(sent)
  return nltktokenizer.tokenize(sent)

In [71]:
from collections import defaultdict

def map_subwords(words, subwords):
  sidx = 0
  widx = 0
  w = ""
  mapping = []

  while widx < len(words):
    w = w + words[widx]
    if sidx >= len(subwords):
      print(widx, w, sidx)
      print(words)
      print(subwords)

    idx = [sidx]
    s = subwords[sidx].replace("#", "")

    while len(w) < len(s):
      widx += 1
      w = w + words[widx].strip()

    # if words[0] == "thotlife":
    #     print(w, s)

    while s!=w and len(s) < len(w) and sidx+1 < len(subwords):
      sidx += 1
      s += subwords[sidx].replace("#", "").strip()
      idx.append(sidx)

      # if words[0] == "thotlife":
      #   print(f"[{w}] [{s}]", len(w), len(s), w==s)

    if len(s) > len(w):
      widx += 1
      sidx -= len(idx) - 1
      continue
    
  
    if len(w) > 20 and w not in words:
      print("ERROR", w)

      # assert(False)    
    sidx += 1
    # print(w, s, idx)
    widx += 1
    w = ""

    mapping.append(idx)

  return mapping

def normalise_quote(tokens):
  for widx, w in enumerate(tokens):
    if w=="``":
      tokens[widx] = '"'
    elif w=="''":
      tokens[widx] = '"'
    elif w=="... ...":
      tokens[widx] = '......'
      
  return tokens

In [72]:
from itertools import groupby
def clean_and_tokenize(sent):
  sent = sent.lower().replace("#", "").replace("''", '"').replace(".", " ").replace("-", " ").replace("&x200b;", "")

  # remove more than 3 consecutive repeated characters
  groups = groupby(sent)
  sent = "".join([label*min(3, sum(1 for _ in group)) for label, group in groups])
  

  sent = sent.encode("ascii", "ignore")
  sent = sent.decode()
  
  words = word_tokenize(sent)
  words = normalise_quote(words)
    
  subwords = swtokenizer.tokenize(sent)
  subwords = normalise_quote(subwords)
  return words, subwords

def merge_score(new_lexicons, idxs, subwords):
  for idx in idxs:
    w = ""
    s = 0
    for i in idx:
      sw = subwords[i].replace("#", "")
      w += sw
      if sw not in lexicons[label]:
      #   print(">>", sw)
        continue

      s += lexicons[label][sw]
      
    new_lexicons[w] = max(new_lexicons[w], s)
  return new_lexicons

In [83]:
def merge_subwords(lexicons, train):
  vocabs = {}
  new_lexicons_scores = {}

  for idx, label in enumerate(label_values):
    print("Working on", label)
    new_lexicons = defaultdict(int)

    d = train[train["label"]==label]
    texts = d["text"].values

    for sent in tqdm(texts, total=len(texts)):
      words, subwords = clean_and_tokenize(sent)
      idxs = map_subwords(words, subwords)
      # new_lexicons = merge_score(new_lexicons, idxs, subwords)
      for idx in idxs:
        w = ""
        s = 0
        for i in idx:
          sw = subwords[i].replace("#", "")
          w += sw
          if sw not in lexicons[label]:
          #   print(">>", sw)
            continue

          s += lexicons[label][sw]
          
        new_lexicons[w] = max(new_lexicons[w], s)
  
    sorted_list = sorted(new_lexicons.items(), key=lambda item: -item[1])
    new_lexicons_scores[label] = { k:v for k, v in sorted_list if v > 0}
  return new_lexicons_scores

newlexicons = merge_subwords(aug_lexicons, data)


1. threats, plans to harm and incitement 1475 ['slutshamed', 'cunts', 'bitches', 'whores', 'bitch', 'pussy', 'whore', 'hoe', 'sluts', 'cunt']
2. derogation 5385 ['noodlewhore', 'bhahahahahaahahahahaha', 'pussies', 'pussypassdenied', 'pussyhats', 'bitches', 'blurred', 'bitchy', 'bitching', 'pussyhat']
3. animosity 4290 ['npcunt', 'pussyfooting', 'muhahahahahahahahahahahahahahahaha', 'twatocracy', 'pussified', 'cuntfused', 'bitchboi', 'pussyboy', 'cuntboys', 'bitches']
4. prejudiced discussions 2029 ['pussies', 'bitches', 'bitch', 'whorellywood', 'feminazis', 'feminazi', 'femoid', 'lesbians', 'pussy', 'jajajajjjajajajaja']


In [86]:
import numpy as np
def filter_lexicons(lexicons, q=0.9):
  threshold = {}
  for l in lexicons:
    values = [v for k, v in lexicons[l].items()]
    threshold[l] = np.quantile(values, q)


  new_lexicons = {}
  for l in lexicons:
    new_lexicons[l] = {}
    for w in lexicons[l]:
      v = lexicons[l][w]

      if v > threshold[l]:
        new_lexicons[l][w] = v
  return new_lexicons

filtered_lexicons = filter_lexicons(newlexicons, q=0.9)

In [87]:
_aug_lexicons = filtered_lexicons
for c in _aug_lexicons:
  _aug_lexicons[c] = {k: v for k, v in sorted(_aug_lexicons[c].items(), key=lambda item: -item[1])}

  print(c, len(_aug_lexicons[c]), list(_aug_lexicons[c].keys())[0:10])

1. threats, plans to harm and incitement 148 ['slutshamed', 'cunts', 'bitches', 'whores', 'bitch', 'pussy', 'whore', 'hoe', 'sluts', 'cunt']
2. derogation 539 ['noodlewhore', 'bhahahahahaahahahahaha', 'pussies', 'pussypassdenied', 'pussyhats', 'bitches', 'blurred', 'bitchy', 'bitching', 'pussyhat']
3. animosity 429 ['npcunt', 'pussyfooting', 'muhahahahahahahahahahahahahahahaha', 'twatocracy', 'pussified', 'cuntfused', 'bitchboi', 'pussyboy', 'cuntboys', 'bitches']
4. prejudiced discussions 203 ['pussies', 'bitches', 'bitch', 'whorellywood', 'feminazis', 'feminazi', 'femoid', 'lesbians', 'pussy', 'jajajajjjajajajaja']


## Evaluation

In [88]:
from sklearn.metrics import f1_score

def predict(word, lexicons):
  cnt = {}
  for label in label_values:
    cnt[label] = 0

  for w in word:
    for label in label_values:
      if w in lexicons[label]:
        cnt[label] += lexicons[label][w]

  
  if sum(cnt.values())==0:
    return None
  
  return max(cnt.items(), key=lambda k: k[1])[0]

def run_predict(test_words, test_labels, lexicons, return_predict=False):
  y_pred = []
  y_test = []
  non = 0
  for word, label in zip(test_words, test_labels):
    pred = predict(word, lexicons)
    if pred is None:
      non += 1
      continue

    y_pred.append(pred)
    y_test.append(label)

  f1 = f1_score(y_test, y_pred, average='macro')
  skip = non/len(test_words)

  if return_predict:
    return f1, skip, y_test, y_pred
    
  return f1, skip

# test = pd.read_csv(f"Data/0_test.csv")
# test_words = [nltk.word_tokenize(sent) for sent in test["text"].values]
# test_labels = test[target_column].values

# run_predict(test_words, test_labels, filtered_lexicons)

In [99]:
from sklearn.metrics import precision_recall_fscore_support
import json


def merge_lexicons(lexicons, aug_lexicons):
  new_lexicons = {}
  for label in label_values:
    new_lexicons[label] = {}
    for word in lexicons[label]:
      new_lexicons[label][word] = lexicons[label][word]

    for word in aug_lexicons[label]:
      if word in new_lexicons[label]:
        new_lexicons[label][word] = max(aug_lexicons[label][word], new_lexicons[label][word])
      else:
        new_lexicons[label][word] = aug_lexicons[label][word]
        
  return new_lexicons


# all_f1 = []
# all_skip = []
# all_p = []
# all_r = []

all_f1, all_skip = {}, {}
for q in np.arange(0, 1, 0.05):
  all_f1[q] = []
  all_skip[q] = []

aug = "shapley"
for i in range(5):
  
  test = pd.read_csv(f"Data/{i}_val.csv")

  texts = test["text"].values
  test_words = [word_tokenize(sent) for sent in test["text"].values]
  test_labels = test[target_column].values

  with open(f'Results/TaskB/lexicon_{aug}_train_{i}.json') as fin:
      lexicons = json.load(fin)
  
  for q in np.arange(0, 1, 0.05):
    filtered_lexicons = filter_lexicons(newlexicons, q=q)
    new_lexicons = merge_lexicons(lexicons, filtered_lexicons)
    # with open(f'Results/TaskB/bertweet_lexicon_{aug}_train_{i}.json', 'w') as outfile:
    #     json.dump(new_lexicons, outfile)

    f1, skip, y_test, y_pred = run_predict(test_words, test_labels, new_lexicons, return_predict=True)
    p, r, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

    # all_p.append(p)
    # all_r.append(r)
    all_f1[q].append(f1)
    all_skip[q].append(skip)

In [100]:
import numpy as np

for q in np.arange(0, 1, 0.05):
  print(f"{q:.3f} >> F1:{np.mean(all_f1[q]):.3f}±{np.std(all_f1[q]):.3f}, SKIP: {np.mean(all_skip[q]):.3f}±{np.std(all_skip[q]):.3f}")


0.000 >> F1:0.450±0.017, SKIP: 0.000±0.000
0.050 >> F1:0.449±0.017, SKIP: 0.000±0.000
0.100 >> F1:0.450±0.016, SKIP: 0.000±0.000
0.150 >> F1:0.445±0.016, SKIP: 0.000±0.000
0.200 >> F1:0.445±0.019, SKIP: 0.000±0.000
0.250 >> F1:0.447±0.017, SKIP: 0.000±0.000
0.300 >> F1:0.431±0.017, SKIP: 0.000±0.000
0.350 >> F1:0.405±0.018, SKIP: 0.000±0.000
0.400 >> F1:0.357±0.013, SKIP: 0.000±0.000
0.450 >> F1:0.367±0.020, SKIP: 0.001±0.001
0.500 >> F1:0.355±0.024, SKIP: 0.001±0.001
0.550 >> F1:0.333±0.016, SKIP: 0.001±0.002
0.600 >> F1:0.305±0.024, SKIP: 0.001±0.002
0.650 >> F1:0.316±0.020, SKIP: 0.001±0.002
0.700 >> F1:0.322±0.018, SKIP: 0.019±0.006
0.750 >> F1:0.320±0.020, SKIP: 0.028±0.007
0.800 >> F1:0.327±0.015, SKIP: 0.041±0.006
0.850 >> F1:0.299±0.018, SKIP: 0.056±0.008
0.900 >> F1:0.295±0.014, SKIP: 0.072±0.012
0.950 >> F1:0.290±0.031, SKIP: 0.089±0.012


In [110]:
all_f1 = []
all_skip = []
all_p = []
all_r = []

aug = "pmi"
for i in range(5):
  
  test = pd.read_csv(f"Data/{i}_test.csv")

  texts = test["text"].values
  test_words = [word_tokenize(sent) for sent in test["text"].values]
  test_labels = test[target_column].values

  with open(f'Results/TaskB/lexicon_{aug}_train_{i}.json') as fin:
      lexicons = json.load(fin)
  
  filtered_lexicons = filter_lexicons(newlexicons, q=0.25)
  new_lexicons = merge_lexicons(lexicons, filtered_lexicons)
  with open(f'Results/TaskB/bertweet_lexicon_{aug}_train_{i}.json', 'w') as outfile:
      json.dump(new_lexicons, outfile)

  f1, skip, y_test, y_pred = run_predict(test_words, test_labels, new_lexicons, return_predict=True)
  p, r, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

  all_p.append(p)
  all_r.append(r)
  all_f1.append(f1)
  all_skip.append(skip)

#### Lexicon Shapley

In [104]:
print(f"F1:{np.mean(all_f1):.3f}±{np.std(all_f1):.3f}, SKIP: {np.mean(all_skip):.3f}±{np.std(all_skip):.3f}")
print(f"P:{np.mean(all_p):.3f}±{np.std(all_p):.3f}, R: {np.mean(all_r):.3f}±{np.std(all_r):.3f}")

F1:0.449±0.021, SKIP: 0.000±0.000
P:0.481±0.024, R: 0.498±0.023


In [105]:
for l in label_values:
  print(l, len(lexicons[l]), len(aug_lexicons[l]), len(new_lexicons[l]))

1. threats, plans to harm and incitement 174 1523 1169
2. derogation 518 4956 4213
3. animosity 435 4046 3364
4. prejudiced discussions 207 2109 1599


#### Lexicon PMI

In [111]:

print(f"F1:{np.mean(all_f1):.3f}±{np.std(all_f1):.3f}, SKIP: {np.mean(all_skip):.3f}±{np.std(all_skip):.3f}")
print(f"P:{np.mean(all_p):.3f}±{np.std(all_p):.3f}, R: {np.mean(all_r):.3f}±{np.std(all_r):.3f}")

F1:0.369±0.017, SKIP: 0.000±0.000
P:0.383±0.014, R: 0.400±0.018


In [112]:
for l in label_values:
  print(l, len(lexicons[l]), len(aug_lexicons[l]), len(new_lexicons[l]))

1. threats, plans to harm and incitement 438 1523 1391
2. derogation 2162 4956 5117
3. animosity 1661 4046 4127
4. prejudiced discussions 538 2109 1829


In [113]:
import json

with open(f'Results/TaskB/lexicon_{aug}.json') as fin:
    lexicons = json.load(fin)

new_lexicons = merge_lexicons(lexicons, aug_lexicons)

with open(f'./Results/TaskB/bertweet_lexicon_{aug}.json', 'w') as outfile:
    json.dump(new_lexicons, outfile)